In [ ]:
%pip install -U git+https://github.com/zalandoresearch/pytorch-ts.git@version-0.7.0

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gluonts.dataset.common import ListDataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

from pts.modules import NegativeBinomialOutput
from pts.model.transformer import TransformerEstimator

In [ ]:
m5_file_path = "~/.gluonts/m5/"

In [ ]:
cal_path = f"{m5_file_path}/calendar.csv"
sales_path = f"{m5_file_path}/sales_train_validation.csv"
sales_test_path = f"{m5_file_path}/sales_train_evaluation.csv"
sell_prices_path = f"{m5_file_path}/sell_prices.csv"

In [ ]:
calendar = pd.read_csv(cal_path, parse_dates=True)
calendar.sort_index(inplace=True)
calendar.date = pd.to_datetime(calendar.date)
time_index = pd.to_datetime(calendar.date)

sales_train_validation = pd.read_csv(
    sales_path,
    index_col=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
)
sales_train_validation.sort_index(inplace=True)

sales_train_evaluation = pd.read_csv(
    sales_test_path,
    index_col=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
)
sales_train_evaluation.sort_index(inplace=True)

sell_prices = pd.read_csv(sell_prices_path, index_col=["item_id", "store_id"])
sell_prices.sort_index(inplace=True)

In [ ]:
sales_train_validation["state"] = pd.CategoricalIndex(
    sales_train_validation.index.get_level_values(5)
).codes
sales_train_validation["store"] = pd.CategoricalIndex(
    sales_train_validation.index.get_level_values(4)
).codes
sales_train_validation["cat"] = pd.CategoricalIndex(
    sales_train_validation.index.get_level_values(3)
).codes
sales_train_validation["dept"] = pd.CategoricalIndex(
    sales_train_validation.index.get_level_values(2)
).codes
sales_train_validation["item"] = pd.CategoricalIndex(
    sales_train_validation.index.get_level_values(1)
).codes

sales_train_evaluation["state"] = pd.CategoricalIndex(
    sales_train_evaluation.index.get_level_values(5)
).codes
sales_train_evaluation["store"] = pd.CategoricalIndex(
    sales_train_evaluation.index.get_level_values(4)
).codes
sales_train_evaluation["cat"] = pd.CategoricalIndex(
    sales_train_evaluation.index.get_level_values(3)
).codes
sales_train_evaluation["dept"] = pd.CategoricalIndex(
    sales_train_evaluation.index.get_level_values(2)
).codes
sales_train_evaluation["item"] = pd.CategoricalIndex(
    sales_train_evaluation.index.get_level_values(1)
).codes

In [ ]:
cardinalities = [sales_train_validation["store"].nunique()]

In [ ]:
train_ds = []

for store_id in range(10):

    store_df = sales_train_validation[sales_train_validation.store == store_id][sales_train_validation.cat== 0]
    time_series = {}

    time_series["start"] = str(time_index[0])
    time_series["item_id"] = f"{store_id}"
    time_series["feat_static_cat"] = [store_id]

    time_series["target"] = store_df.iloc[:,0:1913].values
    train_ds.append(time_series.copy())


In [ ]:
dataset_train = ListDataset(train_ds, freq="D", one_dim_target=False)

In [ ]:
test_ds = []

for store_id in range(10):

    store_df = sales_train_evaluation[sales_train_evaluation.store == store_id][sales_train_evaluation.cat== 0]
    time_series = {}

    time_series["start"] = str(time_index[0])
    time_series["item_id"] = f"{store_id}"
    time_series["feat_static_cat"] = [store_id]

    time_series["target"] = store_df.iloc[:,0:1941].values
    test_ds.append(time_series.copy())


In [ ]:
dataset_test = ListDataset(test_ds, freq="D", one_dim_target=False)

In [ ]:
estimator = TransformerEstimator(
    distr_output=NegativeBinomialOutput(dim=1437),
    input_size=1437,
    lags_seq=[1,],

    d_model=256,
    nhead=8,

    num_encoder_layers=4,
    num_decoder_layers=2,

    prediction_length=28,
    context_length=28*3,
    freq="D",
    scaling=False,
    trainer_kwargs=dict(max_epochs=200, accelerator='gpu', devices='1'),
)

In [ ]:
predictor = estimator.train(dataset_train, cache_data=True)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

In [ ]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

In [ ]:
agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

In [ ]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

In [ ]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

In [ ]:
def plot(target, forecast, prediction_length, prediction_intervals=(50.0, 90.0), color='g', fname=None):
    label_prefix = ""
    rows = 4
    cols = 4
    fig, axs = plt.subplots(rows, cols, figsize=(24, 24))
    axx = axs.ravel()
    seq_len, target_dim = target.shape

    ps = [50.0] + [
            50.0 + f * c / 2.0 for c in prediction_intervals for f in [-1.0, +1.0]
        ]

    percentiles_sorted = sorted(set(ps))

    def alpha_for_percentile(p):
        return (p / 100.0) ** 0.3

    for dim in range(0, min(rows * cols, target_dim)):
        ax = axx[dim]

        target[-2 * prediction_length :][dim].plot(ax=ax)

        ps_data = [forecast.quantile(p / 100.0)[:,dim] for p in percentiles_sorted]
        i_p50 = len(percentiles_sorted) // 2

        p50_data = ps_data[i_p50]
        p50_series = pd.Series(data=p50_data, index=forecast.index)
        p50_series.plot(color=color, ls="-", label=f"{label_prefix}median", ax=ax)

        for i in range(len(percentiles_sorted) // 2):
            ptile = percentiles_sorted[i]
            alpha = alpha_for_percentile(ptile)
            ax.fill_between(
                forecast.index,
                ps_data[i],
                ps_data[-i - 1],
                facecolor=color,
                alpha=alpha,
                interpolate=True,
            )
            # Hack to create labels for the error intervals.
            # Doesn't actually plot anything, because we only pass a single data point
            pd.Series(data=p50_data[:1], index=forecast.index[:1]).plot(
                color=color,
                alpha=alpha,
                linewidth=10,
                label=f"{label_prefix}{100 - ptile * 2}%",
                ax=ax,
            )

    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]
    axx[0].legend(legend, loc="upper left")

    if fname is not None:
        plt.savefig(fname, bbox_inches='tight', pad_inches=0.05)

In [ ]:
plot(
    target=targets[9],
    forecast=forecasts[9],
    prediction_length=28,
)
plt.show()